In [ ]:
# Imports
import pandas as pd
import numpy as np

### Aggregate World Data by Country

In [1]:
# Read and aggregate data
req_cols = ['avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'quarter', 'category', 'iso3', 'name']

for i in range(0,30):
    df = pd.read_csv(f'./data/performance/preprocessed_files/whole_world/whole_world_{i}.csv', sep=';', usecols=req_cols)

    # aggregate as weighted average

    # create helper columns for weighted averages
    df['product1'] = df['avg_d_kbps'] * ['tests']
    df['product2'] = df['avg_u_kbps'] * ['tests']
    df['product3'] = df['avg_lat_ms'] * ['tests']

    # Aggregate
    df_agg = df.groupby('name').agg({'product1': np.average,
                                     'product2': np.average,
                                     'product3': np.average,
                                     'tests': sum})

    # retrieve actual values from product columns
    df_agg['avg_d_kbps'] = df_agg['product1'] / df_agg['tests']
    df_agg['avg_u_kbps'] = df_agg['product2'] / df_agg['tests']
    df_agg['avg_lat_ms'] = df_agg['product3'] / df_agg['tests']

    # drop helper columns
    df = df.drop(columns=['product1', 'product2', 'product3'])

    # Concat together
    df_world = pd.concat([df_world, df])

SyntaxError: invalid syntax (3018605568.py, line 17)

In [22]:
# Quick inspection
df_world.head()

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,quarter,category,iso3,name
0,8763,3646,45,1,1,2019-01-01,fixed,USA,United States of America
1,9195,3347,43,1,1,2019-01-01,fixed,USA,United States of America
2,6833,3788,42,1,1,2019-01-01,fixed,USA,United States of America
3,8895,3429,43,2,2,2019-01-01,fixed,USA,United States of America
4,4877,935,45,3,2,2019-01-01,fixed,USA,United States of America


### Data Exploration

In [ ]:
# Germany
df_germany = pd.read_csv('./data/performance/germany_final_csv', sep=';')

In [25]:
# Basic infos
df_germany.shape

(5347564, 16)

In [26]:
df_germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5347564 entries, 0 to 5347563
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ,            object 
 1   avg_d_kbps   int64  
 2   avg_u_kbps   int64  
 3   avg_lat_ms   int64  
 4   tests        int64  
 5   devices      int64  
 6   quarter      object 
 7   category     object 
 8   long         float64
 9   lat          float64
 10  geometry     object 
 11  iso3         object 
 12  name         object 
 13  continent    object 
 14  region       object 
 15  iso_3166_1_  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 652.8+ MB


In [27]:
df_germany.describe()

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,long,lat
count,5.347564e+06,5.347564e+06,5.347564e+06,5.347564e+06,5.347564e+06,5.347564e+06,5.347564e+06
mean,6.339947e+04,1.869248e+04,2.614932e+01,9.455708e+00,2.921295e+00,9.693960e+00,5.091353e+01
std,6.810522e+04,2.372912e+04,4.398488e+01,2.178635e+01,4.111197e+00,2.105081e+00,1.753226e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,5.866699e+00,4.729041e+01
25%,1.982500e+04,5.951000e+03,1.400000e+01,1.000000e+00,1.000000e+00,8.003540e+00,4.946098e+01
50%,4.446900e+04,1.239000e+04,2.100000e+01,3.000000e+00,2.000000e+00,9.420776e+00,5.103794e+01
75%,8.309300e+04,2.429400e+04,3.000000e+01,9.000000e+00,3.000000e+00,1.129395e+01,5.224126e+01
max,1.616027e+06,9.116170e+05,3.187100e+04,3.987000e+03,5.170000e+02,1.502930e+01,5.505006e+01
